In [ ]:
#time series date vs. sentiment score -all the reviews vs. top 5 reviewed branches
#same graph for all the years

In [1]:
#split the date part to year and month
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import make_interp_spline

In [ ]:
#Read the dataframe with predicted sentiment scores
LLM_predicts = pd.read_json('llm_sentiment.json', lines=True)
BOW_predicts = pd.read_json('BOW_predictions.json', lines=True)

In [ ]:
grouped = LLM_predicts.groupby(['text', 'llm_sentiment']).size()
(
    grouped
    .reset_index(name='count')
    ["text"]
    .value_counts()
    .reset_index(name='count')
    .query('count > 1')
    .any()
)

#.query and how many are grater than 1


text     False
count    False
dtype: bool

In [15]:
LLM_predicts.drop_duplicates(subset=['text'], keep='first', inplace=False)
LLM_predicts

,business_id,text,llm_sentiment
0,aJvxWyQIG5OLfBw3qAe8xA,"Ordered Caramel frappe at Drive thru, BIG MIST...",1
1,MjZQqZAmJeMco_Vq-Y9h-g,Drum-roll please! Review #100 coming right up!...,3
2,u7MJKcNdZXYyTeb67vD5jw,We stopped here for my Chai and Hubby's coffee...,3
3,saJFbz12EnzanelpD8_xXQ,There's been three times that I've ordered a g...,1
4,KiE0h68HGOO7ZXAqkMBdiw,"I went in when they had 4 people working, wait...",1
...,...,...,...
21570,-85kJMtb9wqNWDT8yLbitw,Fair service- messed up my drink cuppa times....,1
21571,Ddg-J_j0YFErk7wpMtH_0A,On my way out of Reno last week I noted a new ...,3
21572,2rmpfdyV2POqpXtmPHO_IQ,always closing drive thru. i've driven here so...,1
21573,GxuxCctcz3Hyk0wnuly7vQ,This is now one of two Starbucks in the shoppi...,3


In [5]:
v = LLM_predicts["text"].value_counts()
v

text
Basic Starbucks experience. Nothing fancy. Had the regular items. Fairly small seating area. Had an outdoor seating but it was too cold to sit outside today. Friendly service.                                                                                                                                                                                                                                                                                                                                                                               3
Every Starbucks is the same bland, boring, sterile yuppie infested, "I'm in a rush, get my latte quick" type place.\n\nAwful Coffee, lousy rude service, terrible sandwiches, overpriced everything.\n\nI avoid it always unless I get a free gift card.                                                                                                                                                                                                           

In [16]:
dataset_df_final = pd.merge(LLM_predicts,BOW_predicts, on="text")
dataset_df_final

,business_id_x,text,llm_sentiment,user_id,business_id_y,stars,useful,funny,cool,date,name,review_count,categories,address,city,state,postal_code,stars_3_scale,bow_predicted_sentiment
0,aJvxWyQIG5OLfBw3qAe8xA,"Ordered Caramel frappe at Drive thru, BIG MIST...",1,veBX5roHQLNnQ1rTtNh_gg,aJvxWyQIG5OLfBw3qAe8xA,2,0,0,0,2017-12-29 19:38:31,Starbucks,65,"Restaurants, Coffee & Tea, Food",2308 Murfreesboro Pike,Nashville,TN,37217,1,1
1,MjZQqZAmJeMco_Vq-Y9h-g,Drum-roll please! Review #100 coming right up!...,3,zZygOZLNyJX5nDqKrwa4HQ,MjZQqZAmJeMco_Vq-Y9h-g,4,0,0,0,2014-02-05 19:38:24,Starbucks,33,"Food, Coffee & Tea",13170 North Hazel Dell Pkwy,Carmel,IN,46033,3,3
2,u7MJKcNdZXYyTeb67vD5jw,We stopped here for my Chai and Hubby's coffee...,3,0SpK7eVn5TrpyQKPaohVBw,u7MJKcNdZXYyTeb67vD5jw,4,1,0,0,2017-02-09 04:35:39,Starbucks,17,"Food, Coffee & Tea",5 Hartford Rd,Mount Laurel,NJ,08054,3,3
3,saJFbz12EnzanelpD8_xXQ,There's been three times that I've ordered a g...,1,IVnrNZ8npRx04jkNp6-ERA,saJFbz12EnzanelpD8_xXQ,2,0,0,0,2016-08-25 14:08:18,Starbucks,95,"Food, Coffee & Tea",10716 Big Bend Rd,Riverview,FL,33579,1,3
4,KiE0h68HGOO7ZXAqkMBdiw,"I went in when they had 4 people working, wait...",1,tgP2gKsf3O9MyuLuolv2oQ,KiE0h68HGOO7ZXAqkMBdiw,1,0,1,0,2016-01-30 01:10:42,Starbucks,35,"Food, Coffee & Tea",1015 De La Vina St,Santa Barbara,CA,93101,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21706,-85kJMtb9wqNWDT8yLbitw,Fair service- messed up my drink cuppa times....,1,QPDFn3SQqZeBu7OBI78g9w,-85kJMtb9wqNWDT8yLbitw,3,0,0,0,2021-12-13 22:08:14,Starbucks,34,"Food, Coffee & Tea","438 Main St, Space 168, Space 168",Franklin,TN,37064,2,1
21707,Ddg-J_j0YFErk7wpMtH_0A,On my way out of Reno last week I noted a new ...,3,F3ELBqGgDCX9YEhz_oS0xQ,Ddg-J_j0YFErk7wpMtH_0A,5,3,1,4,2021-12-28 19:26:37,Starbucks,67,"Food, Coffee & Tea, Coffee Roasteries",986 Ambassador Dr,Reno,NV,89523,3,3
21708,2rmpfdyV2POqpXtmPHO_IQ,always closing drive thru. i've driven here so...,1,TZJzzQsE4R7RmqkVRaLEKw,2rmpfdyV2POqpXtmPHO_IQ,1,0,0,0,2021-12-28 18:12:04,Starbucks,27,"Coffee & Tea, Food",6363 N La Cholla,Tucson,AZ,85741,1,1
21709,GxuxCctcz3Hyk0wnuly7vQ,This is now one of two Starbucks in the shoppi...,3,dvu83QWKsnnbGvNVQ3mv7A,GxuxCctcz3Hyk0wnuly7vQ,4,1,0,1,2017-01-21 00:19:27,Starbucks,45,"Food, Coffee & Tea",4811 E Grant,Tucson,AZ,85712,3,3


In [ ]:
BOW_predicts["year"] = BOW_predicts["date"].dt.year
BOW_predicts["month"] = BOW_predicts["date"].dt.month
BOW_predicts["day"] = BOW_predicts["date"].dt.day
BOW_predicts

In [ ]:
#gruoup by day city, rolling average by 7 days
BOW_predicts = BOW_predicts.groupby(['day', 'city']).agg({'bow_predicted_sentiment': 'mean'}).reset_index()

In [ ]:
# Group by day and city, compute mean sentiment_score
grouped = BOW_predicts.groupby(['date', 'city']).agg({'bow_predicted_sentiment': 'mean'}).reset_index()

# Ensure day is datetime and sort for correct rolling
grouped['date'] = pd.to_datetime(grouped['date'])
grouped = grouped.sort_values('date')

# Set day as index for rolling
grouped = grouped.set_index('date')

# Calculate 7-day rolling average for each city
result = grouped.groupby('city')['bow_predicted_sentiment'].rolling('7D').mean().reset_index()


In [ ]:
result

In [ ]:
v = BOW_predicts[['city', 'year']].value_counts().sort_values(ascending=False)
v

In [ ]:
v = BOW_predicts['bow_predicted_sentiment'].value_counts()
v

In [ ]:
BOW_predicts["year"].unique()

In [ ]:
import plotly.express as px

In [ ]:
year = 2021
cities = ['Philadelphia', 'Indianapolis', 'Tampa', 'Tucson', 'Reno']  # replace with your actual city names
filtered_df = BOW_predicts[(BOW_predicts["year"] == year) & (BOW_predicts["city"].isin(cities))]


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ensure date is datetime and extract month and year
BOW_predicts['date'] = pd.to_datetime(BOW_predicts['date'])
BOW_predicts['month'] = BOW_predicts['date'].dt.to_period('M')
BOW_predicts['year'] = BOW_predicts['date'].dt.year

# Filter for years 2016 to 2021
filtered_df = BOW_predicts[(BOW_predicts['year'] >= 2016) & (BOW_predicts['year'] <= 2021)]

# Group by month and sentiment, count occurrences
sentiment_counts = filtered_df.groupby(['month', 'bow_predicted_sentiment']).size().unstack(fill_value=0)

# Calculate percentage for each sentiment per month
sentiment_percent = sentiment_counts.div(sentiment_counts.sum(axis=1), axis=0) * 100

# Plot as stacked bar chart
sentiment_percent.plot(
    kind='bar',
    stacked=True,
    color=['#d62728', '#ff7f0e', '#2ca02c'],  # red=negative, orange=neutral, green=positive
    figsize=(12, 6)
)
plt.ylabel('Percentage of Reviews')
plt.xlabel('Month')
plt.title('Percentage allocation of reviews (2016–2021)')
plt.legend(title='Sentiment')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Ensure date is datetime and extract month and year
BOW_predicts['date'] = pd.to_datetime(BOW_predicts['date'])
BOW_predicts['month'] = BOW_predicts['date'].dt.to_period('M')
BOW_predicts['year'] = BOW_predicts['date'].dt.year

# Filter for years 2016 to 2021
filtered_df = BOW_predicts[(BOW_predicts['year'] >= 2016) & (BOW_predicts['year'] <= 2021)]

# Group by month and sentiment, count occurrences
sentiment_counts = filtered_df.groupby(['month', 'bow_predicted_sentiment']).size().unstack(fill_value=0)

# Calculate percentage for each sentiment per month
sentiment_percent = sentiment_counts.div(sentiment_counts.sum(axis=1), axis=0) * 100

# Plot as stacked bar chart
sentiment_percent.plot(
    kind='line',
    #stacked=True,
    color=['#d62728', '#ff7f0e', '#2ca02c'],  # red=negative, orange=neutral, green=positive
    figsize=(12, 6)
)
plt.ylabel('Percentage of Reviews')
plt.xlabel('Month')
plt.title('Percentage allocation of reviews (2016–2021)')
plt.legend(title='Sentiment')
plt.tight_layout()
plt.show()


In [ ]:
# Ensure 'month' is sorted and numeric
year = 2020
BOW_predicts['date'] = pd.to_datetime(BOW_predicts['date'])
BOW_predicts = BOW_predicts.sort_values('date')
cities = ['Philadelphia', 'Indianapolis', 'Tampa', 'Tucson', 'Reno']
filtered_df = BOW_predicts[(BOW_predicts["year"] == year) & (BOW_predicts["city"].isin(cities))]

fig = px.line(
    filtered_df,
    x='date',
    y='bow_predicted_sentiment',
    color='city',
    markers=True,
    labels={'date': 'date', 'bow_predicted_sentiment': 'Value', 'city': 'City'},
    title=f'Monthly Data for Selected Cities in {year}'
)

fig.update_layout(
    xaxis_title='date',
    yaxis_title='Value',
    legend_title='City',
    plot_bgcolor='white'
)

fig.show()
